## Ejemplo de red cerrada por el método de la teoría lineal

Se resuelve, por medio del método de teoróa lineal, el problema 6.10 del libro *Mecánica de fluidos e hidráulica* del profesor Carlos A. Duarte de la Universidad Nacional de Colombia (segunda edición). Se considerará que cada tubería tiene una rugosidad de 0.03 mm y se hallarán los factores de fricción en cada una de las iteraciones para comprobar la respuesta que da el libro, usando los coeficientes C que no permiten que haya variaciones de los factores $f$ de Darcy. 

In [1]:
import numpy as np
import Funciones as funct
import pandas as pd

In [2]:
# VARIABLES GEOMÉTRICAS DE LA RED DESCRITA. 
Tuberias = ['AB', 'BC', 'CD', 'DE', 'EA', 'BE']
Diam_mm = [200, 150, 100, 150, 200, 100]
Long_m = [500, 600, 200, 600, 600, 200]
Rugos_mm = [0.03] * 6

# Volviendo las variables un dataframe
Tuberias_df = pd.DataFrame({
    'Tuberia': Tuberias,
    'Diametro_mm': Diam_mm,
    'Longitud_m': Long_m,
    'Rugosidad_mm': Rugos_mm
})

Tuberias_df['ks/D'] = Tuberias_df['Rugosidad_mm'] / Tuberias_df['Diametro_mm']
Tuberias_df['Diametro_m'] = Tuberias_df['Diametro_mm'] / 1000
Tuberias_df

,Tuberia,Diametro_mm,Longitud_m,Rugosidad_mm,ks/D,Diametro_m
0,AB,200,500,0.03,0.00015,0.20
1,BC,150,600,0.03,0.00020,0.15
2,CD,100,200,0.03,0.00030,0.10
3,DE,150,600,0.03,0.00020,0.15
4,EA,200,600,0.03,0.00015,0.20
5,BE,100,200,0.03,0.00030,0.10


In [3]:
# Declarando arreglos que contienen las tuberías que conforman cada uno de los 
# circuitos
CircuitoI = ['AB', 'EA', 'BE']
CircuitoII = ['BC', 'CD', 'DE', 'BE']

# Filtrando las tuberías que conforman cada circuito
Tuberias_CircuitoI = Tuberias_df[
    Tuberias_df['Tuberia'].isin(CircuitoI)].reset_index(drop=True)
Tuberias_CircuitoII = Tuberias_df[
    Tuberias_df['Tuberia'].isin(CircuitoII)].reset_index(drop=True)

Tuberias_CircuitoII

,Tuberia,Diametro_mm,Longitud_m,Rugosidad_mm,ks/D,Diametro_m
0,BC,150,600,0.03,0.0002,0.15
1,CD,100,200,0.03,0.0003,0.10
2,DE,150,600,0.03,0.0002,0.15
3,BE,100,200,0.03,0.0003,0.10


In [4]:
# Como debo resolver un sistema de 6x6, se crea una matriz de coeficentes para 
# la colución del mismo. Se pondrán las ecuaciones de conservación de masa en 
# los nodos en las primeras filas (son 4 ecuaciones), y las ecuaciones de 
# conservación de energía en las últimas dos filas (una por circuito).

Mat_coefic = np.zeros((len(Tuberias), len(Tuberias)), dtype=float)

# Agrego a mano las ecauciones de conservación de masa en los nodos:
# Nodo B:
Mat_coefic[0, 0] = -1
Mat_coefic[0, 1] = 1
Mat_coefic[0, 5] = 1

# Nodo C:
Mat_coefic[1, 1] = -1
Mat_coefic[1, 2] = 1

# Nodo D:
Mat_coefic[2, 2] = -1
Mat_coefic[2, 3] = 1

# Nodo E:
Mat_coefic[3, 3] = -1
Mat_coefic[3, 4] = 1
Mat_coefic[3, 5] = -1

# Imprimiendo la matriz de coeficientes hasta el momento:
print("Matriz de coeficientes después de agregar las ecuaciones " +
 "de conservación de masa:")
Mat_coefic

Matriz de coeficientes después de agregar las ecuaciones de conservación de masa:


array([[-1.,  1.,  0.,  0.,  0.,  1.],
       [ 0., -1.,  1.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  1.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  1., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.]])

In [5]:
# Ahora se definirá una función para poder llenar las ecuaciones de conservación 
# de la energía en cada uno de los circuitos. La función recibe la matriz A, un 
# vector con caudales supuestos y el dataframe de las tuberías. 
# Devuelve la matriz A llena con las ecuaciones.
def llenar_matriz(A, Q, Tub):

    # Agrego una columna al dataframe con los caudales, con esta hallo RE y 
    # luego los factores de fricción. Por último, otra que calcule los 
    # coeficientes de la pérdida de carga
    Tub['Q'] = Q
    Tub['Re'] = 4 * np.abs(Tub['Q']) / (np.pi * Tub['Diametro_m'] * 1.14e-6)
    Tub['f'] = funct.Col_Whi(Tub['Diametro_mm'], Tub['Rugosidad_mm'],
    Tub['Re'])
    Tub['K'] = 0.0826 * Tub['f'] * Tub['Longitud_m'] / Tub['Diametro_m'] ** 5

    # Ecuación para el circuito I (fila 4): 
    A[4, 0] = Tub['K'][0] * np.abs(Tub['Q'][0])
    A[4, 4] = Tub['K'][4] * np.abs(Tub['Q'][4])
    A[4, 5] = Tub['K'][5] * np.abs(Tub['Q'][5])

    # Ecuación para el circuito II (fila 5):
    A[5, 1] = Tub['K'][1] * np.abs(Tub['Q'][1])
    A[5, 2] = Tub['K'][2] * np.abs(Tub['Q'][2])
    A[5, 3] = Tub['K'][3] * np.abs(Tub['Q'][3])
    A[5, 5] = -Tub['K'][5] * np.abs(Tub['Q'][5])

    # Devolviendo la matriz A completa:
    return A

# Haciendo una prueba del llenado de la matriz con un vector cualquiera de 
# caudales
np.set_printoptions(precision=4, suppress=True)
llenar_matriz(Mat_coefic, [0.1] * 6, Tuberias_df)

array([[   -1.    ,     1.    ,     0.    ,     0.    ,     0.    ,
            1.    ],
       [    0.    ,    -1.    ,     1.    ,     0.    ,     0.    ,
            0.    ],
       [    0.    ,     0.    ,    -1.    ,     1.    ,     0.    ,
            0.    ],
       [    0.    ,     0.    ,     0.    ,    -1.    ,     1.    ,
           -1.    ],
       [  190.9418,     0.    ,     0.    ,     0.    ,   229.1302,
         2576.0652],
       [    0.    ,   976.2173,  2576.0652,   976.2173,     0.    ,
        -2576.0652]])

In [ ]:
np.set_printoptions(precision=5, suppress=True)
# Ahora defino el vector de mano derecha para la solución del sistema de 
# ecuaciones
b = np.array([-0.06, -0.05, -0.04, -0.05, 0., 0.])

# Ahora, se supone un vector de caudales iniciales para resolver el sistema, se
# resuelve, se imprime el resultado y se actualiza la solución hasta que llegue 
# a una condición estable
err = 1e10
tol = 1e-6
count = 1

Q0 = np.array([0.1] * 6, dtype=float)

while err > tol:

    # Llenando la matriz de coeficientes con los caudales actuales
    Mat_coefic = llenar_matriz(Mat_coefic, Q0, Tuberias_df)

    # Resolviendo el sistema de ecuaciones
    Q1 = np.linalg.solve(Mat_coefic, b)

    # Calculando el error entre la solución actual y la anterior
    err = np.max(np.abs(Q1 - Q0))

    # Imprimiendo valores hallados y su error
    print(f"\nIteración: {count}\n Caudales: {Q1}\n Error: {err:.6f}")

    # Actualizando el vector de caudales para la siguiente iteración
    Q0 = 0.5 * (Q1 + Q0)

    # Incrementando el contador de iteraciones
    count += 1 


Iteración: 1
 Caudales: [ 0.1081   0.04794 -0.00206 -0.04206 -0.0919   0.00016]
 Error: 0.191902

Iteración: 2
 Caudales: [ 0.08281  0.03367 -0.01633 -0.05633 -0.11719 -0.01086]
 Error: 0.121243

Iteración: 3
 Caudales: [ 0.09007  0.03183 -0.01817 -0.05817 -0.10993 -0.00176]
 Error: 0.053360

Iteración: 4
 Caudales: [ 0.10373  0.0424  -0.0076  -0.0476  -0.09627  0.00133]
 Error: 0.013022

Iteración: 5
 Caudales: [ 0.10531  0.04539 -0.00461 -0.04461 -0.09469 -0.00008]
 Error: 0.007576

Iteración: 6
 Caudales: [ 0.1049   0.04538 -0.00462 -0.04462 -0.0951  -0.00048]
 Error: 0.003374

Iteración: 7
 Caudales: [ 0.1048   0.04542 -0.00458 -0.04458 -0.0952  -0.00061]
 Error: 0.001632

Iteración: 8
 Caudales: [ 0.10477  0.04543 -0.00457 -0.04457 -0.09523 -0.00066]
 Error: 0.000859

Iteración: 9
 Caudales: [ 0.1048   0.04542 -0.00458 -0.04458 -0.0952  -0.00062]
 Error: 0.000406

Iteración: 10
 Caudales: [ 0.10481  0.04541 -0.00459 -0.04459 -0.09519 -0.0006 ]
 Error: 0.000217

Iteración: 11
 Cau

In [7]:
# Agregando los caudales hallados al dataframe de tuberías: 
Tuberias_df['Q_m3/s'] = Q1
Tuberias_df['Q_l/s'] = np.round(Tuberias_df['Q_m3/s'] * 1000, 2)

# Imprimiendo el dataframe final con los caudales hallados
print("\nDataframe final con los caudales hallados:")
Tuberias_df[['Tuberia', 'Q_m3/s', 'Q_l/s']].round(4)


Dataframe final con los caudales hallados:


,Tuberia,Q_m3/s,Q_l/s
0,AB,0.1048,104.81
1,BC,0.0454,45.41
2,CD,-0.0046,-4.59
3,DE,-0.0446,-44.59
4,EA,-0.0952,-95.19
5,BE,-0.0006,-0.60
